# Maximum Likelihood Estimation

This notebook illustrates maximum likelihood and how to estimate different standard errors (from the information matrix, the gradients and the "sandwich" approach).

The application is very basic: estimate the mean and variance of a series.

## Loading Packages

In [1]:
using Dates, LinearAlgebra, DelimitedFiles, Statistics, Optim, ForwardDiff

include("jlFiles/printmat.jl")

printlnPs (generic function with 2 methods)

## Loading Data

In [2]:
xx  = readdlm("Data/FFdSizePs.csv",',',skipstart=1)
x   = xx[:,2]                 #returns for the smallest size portfolio
xx  = nothing

## Traditional Estimates

of the mean $\mu$ and the variance $\sigma^2$.

To compare with the MLE, we use $1/T$ in variance estimate, not $1/(T-1)$.

In [3]:
T = length(x)

(μ_trad,σ²_trad) = (mean(x),var(x,corrected=false))

std_trad = sqrt.([σ²_trad,2*σ²_trad^2]/T)          #standard errors, textbook formulas

println("Traditional estimate and std:")
println("  estimate      std")
printmat([[μ_trad,σ²_trad] std_trad])

Traditional estimate and std:
  estimate      std
     0.042     0.010
     0.840     0.013



# Point Estimates from ML

## The Likelihood Function for Estimating the Parameters of a N(,)

In [4]:
function NormalLL(par::Vector,x)
  (μ,σ²) = par
  LLt    = -(1/2)*log(2*pi) .- (1/2)*log.(σ²) .- (1/2)*(x.-μ).^2/σ²  #vector, all x[t]
  loss   = -sum(LLt)        #scalar, to minimize
  return loss, LLt
end

NormalLL (generic function with 1 method)

## Try the Likelihood Function

In [5]:
par0 = [0.0,1.0]                #initial parameter guess

(loss,LLt) = NormalLL(par0,x)   #trying the log likelihood fn

println("log likelohood value at par0: ",-loss)

log likelohood value at par0: -11155.385338928902


## Optimize the Likelihood Function

In [6]:
Sol = optimize(par->NormalLL(par,x)[1],par0)  #minimize -sum(LLt)

parHat = Optim.minimizer(Sol)                 #the optimal solution 

printlnPs("log-likelihood at point estimate: ",-Optim.minimum(Sol))

println("\nParameter estimates: ")
println("   Traditional  MLE  ")
printmat([[μ_trad,σ²_trad] parHat])

log-likelihood at point estimate: -11088.409

Parameter estimates: 
   Traditional  MLE  
     0.042     0.042
     0.840     0.840



# Standard Errors I: Information Matrix 

If the likelihood function is correctely specified, then MLE is typically asymptotically normally distributed as

$
\sqrt{T}(\hat{\theta}-\theta)  \rightarrow^{d}N(0,V) \: \text{, where } \: V=I(\theta)^{-1}\text{ with }
$

$
I(\theta) =-\text{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}}
$

where $I(\theta)$ is the information matrix and $\ln L_t$  is the contribution of period $t$ to the likelihood function.

The code below calculates numerical derivatives and uses the fact that 
$
\text{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}} = 
\frac{\partial^{2}\text{E}\ln L_t}{\partial\theta\partial\theta^{\prime}}
$.

In [7]:
              #use the fact that E(derivative) = derivative(E)
Ia       = -ForwardDiff.hessian(par->mean(NormalLL(par,x)[2]),parHat)
Ia       = (Ia+Ia')/2         #to guarantee symmetry, fixes rounding errors
vcv      = inv(Ia)/T
std_hess = sqrt.(diag(vcv))

println("standard errors:")
println("   traditional Hessian ")
printmat([std_trad std_hess ])

standard errors:
   traditional Hessian 
     0.010     0.010
     0.013     0.013



# Standard Errors II: Gradients and Sandwich

Alternatively, we can use the outer product of the gradients to calculate the
information matrix as

$
J(\theta)=\text{E}\left[  \frac{\partial\ln L_t}{\partial\theta
}\frac{\partial\ln L_t}{\partial\theta^{\prime}}\right]
$

We could also use the "sandwich" estimator

$
V=I(\theta)^{-1}J(\theta)I(\theta)^{-1}.
$

When data is *not* iid $N($), then the three variance-covariance matrices may differ, and the sandwich approach is often the most robust.

### Std from Gradients

In [8]:
LL_t_grad = ForwardDiff.jacobian(par->NormalLL(par,x)[2],parHat)   #T x 2 matrix
J         = LL_t_grad'LL_t_grad/T
vcv       = inv(J)/T
std_grad  = sqrt.(diag(vcv))                          #std from gradients

println("standard errors")
println("    traditional Hessian  gradients")
printmat([std_trad std_hess std_grad])

standard errors
    traditional Hessian  gradients
     0.010     0.010     0.010
     0.013     0.013     0.005



### Std from Sandwich

In [9]:
vcv       = inv(Ia) * J * inv(Ia)/T
std_sandw = sqrt.(diag(vcv))                          #std from sandwich

println("standard errors")
println("    traditional Hessian  gradients sandwich")
printmat([std_trad std_hess std_grad std_sandw])

standard errors
    traditional Hessian  gradients sandwich
     0.010     0.010     0.010     0.010
     0.013     0.013     0.005     0.036



Try this: replace the data series `x` with simulated data from a $N()$ distribution. Do the different standard errors to get closer to each other?